---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [15]:
sigma = 50                                                         # volatility in basis points
prds_per_year = 2
num_bonds = 6                                                      # number of bonds
maturities   = [1, 2, 3, 4, 5, 6]                                  # bond maturities
coupon_rates = [0.02, 0.02, 0.02, 0.02, 0.02, 0.02]                # coupon rates (annual)
yields       = [0.015, 0.02, 0.0225, 0.024, 0.025, 0.026]         # yields (annual)

if not all(len(lst)==num_bonds for lst in [maturities, coupon_rates, yields]):
    print('Lists need to be of the same length!')

# corporate bond information
coupon_rate = 0.05   # annual coupon rate, coupons paid semi-annually
maturity = 10        # maturity in number of periods
option = "put"       # use "put" or "call"
strike = 100
bond_price = 102     # given bond price

if (bond_price<=100 and option=="put") or (bond_price>=100 and option=="call"):
    print("Error: Option should have already been exercised.")

# FUNCTIONS FOR BOND PRICES AND FORWARD RATES

Unlike the "spot and forward rates" notebook, here we work with a general number of periods per year rather than six-month periods.

In [16]:
import numpy as np
from scipy.optimize import minimize

# calculate bond price for given yield to maturity or for given spot rates
def price(**kwargs):
    c = 100 * kwargs['coupon'] / 2
    maturity = kwargs['maturity']
    if 'yld' in kwargs.keys():
        yld = kwargs['yld']
        n = int(2 * maturity)
        return c * np.sum((1 + yld/2) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld/2) ** n
    else:
        n = int(prds_per_year * maturity)
        pphy = int(prds_per_year/2)  # periods per half year
        spots = np.array(kwargs['spots'])
        pv_factors = (1 + spots / prds_per_year) ** (-np.arange(1, len(spots) + 1))
        coupons = np.zeros(n)
        coupons[(pphy - 1)::pphy] = c
        return np.sum(coupons*pv_factors[:len(coupons)]) + 100*pv_factors[n-1]
    
# forward rates implied by spot rates
def forward_rates(spots):
    pphy = int(prds_per_year / 2)  # periods per half year
    future_factors = (1 + spots / 2) ** (np.arange(1, len(spots) + 1) / pphy)
    diff_logs = np.diff(np.log(future_factors))
    f = (np.exp(diff_logs)-1) * prds_per_year
    return np.concatenate(([spots[0]], f))

# FUNCTION TO COMPUTE SPOT RATES

We choose spot rates to match bond prices as closely as possible, except that we add a penalty for variation in implied forward rates, so the resulting forward rate curve is reasonably smooth.  

In [17]:
# difference between bond price and price implied by spot rates
def error(bond, spots):
    price1 = price(**bond)
    price2 = price(maturity=bond["maturity"], coupon=bond['coupon'], spots=spots)
    return np.log(price2 / price1)

# sum of squared errors between bond prices and prices implied by spot rates
# plus a penalty for variation in forward rates
def objective(bonds, spots):
    errors = [error(bond, spots) for bond in bonds]
    sse = np.sum(np.array(errors)**2)
    forwards = forward_rates(spots)
    diffs = np.sum(np.diff(np.log(1+forwards/prds_per_year))**2)
    return sse + 0.5*diffs

# choose spot rates to optimize the objective
def spot_rates(bonds):
    maturities = [bond['maturity'] for bond in bonds]
    n = int(np.max(maturities) * prds_per_year)
    result = minimize(lambda x: objective(bonds, x), [0.05]*n)
    return result.x if result.success==True else np.nan

# SPOT AND FORWARD RATES

In [18]:
bonds = [
    dict(
        maturity= maturities[i],
        coupon  = coupon_rates[i],
        yld     = yields[i]
    )
    for i in range(num_bonds)
]

spots    = spot_rates(bonds)
forwards = forward_rates(spots)

# INTEREST RATE TREE

We find the drift of the interest rate tree so that it matches the forward rates calculated above.  See, for example, Back, A Course in Derivative Securities, 2006.

In [19]:
n = prds_per_year * np.max(maturities)
dt = 1 / prds_per_year
m = len(forwards)
s = sigma / 10000   # volatility as decimal
f = forwards[:n+1] if m>=n+1 else np.concatenate((forwards, [forwards[-1]]*(n+1-m)))
    
term1 = np.log(1+f*dt)
a = np.exp(s*dt**(3/2)*np.arange(n+1))
term2 = np.log(a + 1/a)
term3 = np.log(2*(1+f[0]*dt))
phis = (term1 + term2 - term3) / dt

delta = sigma*np.sqrt(dt) / 10000
rate_tree =  [
    [spots[0] + phi + delta * (i - 2 * j) for j in range(i + 1)] 
    for i, phi in enumerate(phis)
]

# PRICE A BOND WITH OR WITHOUT EMBEDDED OPTION FROM A RATE TREE

In [20]:
def BondTree(tree, kind=None):

    pphy = int(prds_per_year / 2)  # periods per half year

    # coupons paid semi-annually, 0 at other periods
    coupon = coupon_rate * 100 / 2
    coupons = np.zeros(maturity+1)
    coupons[pphy::pphy] = coupon
   
    rates = [np.array(x) for x in tree]

    # x = dirty price, p = clean price
    # initialize tree at maturity
    x = (100+coupon) * np.ones(maturity + 1)
    p = 100 * np.ones(n+1)
    lstp = [p]

    # back up in the tree
    for i in range(maturity-1, -1, -1):
        j = (i % pphy) / pphy
        accrued = (j*coupon) if j>0 else (coupon if i>0 else 0)
        x = accrued + 0.5 * (x[:-1] + x[1:]) / (1 + rates[i] / prds_per_year)

        # consider early exercise
        if kind:
            x = (
                np.minimum(x, strike+accrued) if kind == "call" else
                np.maximum(x, strike+accrued)
            )
        
        p = x - accrued
        lstp.insert(0, p)
    
    # replace subsequent values with strike if exercised early
    for i, arr in enumerate(lstp[:-1]):
        for j in range(len(arr)):
            if arr[j] == strike:
                lstp[i+1][j] = lstp[i+1][j+1] = strike
                
    return [list(p) for p in lstp]

# OPTION-ADJUSTED SPREAD

Calculate spread over rate tree that equates tree-implied bond price to given price.

In [21]:
from scipy.optimize import root_scalar

def error(oas) :
    oas_tree = [[oas+x for x in lst] for lst in rate_tree]
    bond_tree = BondTree(oas_tree, kind=option)
    return bond_price - bond_tree[0][0]

try: 
    oas = root_scalar(error, bracket=[-1, 1], method="brentq")
    oas = oas.root
    print(f"Option Adjusted Spread is {oas:.2%}")
    
except:
    print("failed to solve for OAS")

Option Adjusted Spread is 2.35%


# FIGURES

If the solution for the OAS converged, we generate four figures: 

- "rate" is the short rate tree
- "oas" adds the OAS to the short rate
- "stripped" is the bond value stripped of the option
- "bond" is the bond value including the option

In [22]:
import plotly.graph_objects as go

oas_tree = [[oas+x for x in lst] for lst in rate_tree]
trees = {
    "rate": rate_tree,
    "oas": oas_tree,
    "stripped": BondTree(oas_tree),
    "bond": BondTree(oas_tree, kind=option)
}

titles = {
    "rate": "Annualized Short Rate",
    "oas": "Short Rate + OAS",
    "stripped": "Bond w/o Option",
    "bond": "Bond with Option"
}

figs = {}
for key, tree in trees.items():
    string = "%{y:.2%}" if key in ("rate", "oas") else "$%{y:,.2f}"
    string += "<extra></extra>"
    spliced = []
    for a, b in zip(tree[1:], tree[:-1]):
        x = []
        for i in range(len(a)):
            x.append(a[i])
            try:
                x.append(b[i])
            except:
                pass
        spliced.append(x)
    fig = go.Figure()
    for i in range(len(tree) - 1):
        x = [1, 0, 1]
        for j in range(i):
            x.append(0)
            x.append(1)
        x = np.array(x) + i
        y = spliced[i]
        trace = go.Scatter(
            x=x*dt,
            y=y,
            mode="lines+markers",
            hovertemplate=string,
            marker=dict(size=10, color="blue"),
            line_color="blue",
        )
        fig.add_trace(trace)
    fig.update_layout(
        xaxis_title="Time (years)",
        yaxis_title=titles[key],
        yaxis_tickprefix = None if key in ("rate", "oas") else "$", 
        yaxis_tickformat=".2%" if key in ("rate", "oas") else ".0f",
        yaxis_autorange=None if key in ("rate", "oas") else "reversed",
        template="plotly_white",
        showlegend=False
    )
    figs[key] = fig

In [23]:
# SHORT RATE

In [24]:
figs["rate"].show()

# SHORT RATE PLUS OAS

In [25]:
figs["oas"].show()

# BOND VALUE STRIPPED OF OPTION

In [26]:
figs["stripped"].show()

# BOND VALUE

In [27]:
figs["bond"].show()